<a href="https://colab.research.google.com/github/Mors666/hopfieldRNN/blob/master/PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import numpy as np
from sympy import KroneckerDelta as kron
import matplotlib.pyplot as plt
#import pydot
import time
import random

In [0]:
torch.set_printoptions(threshold=5000)

In [22]:
n = torch.LongTensor([30]) #общее число работников в организации

C = np.random.randint(0, 11, (n, n)) #затраты ребер графа
C = np.tril(C) + np.tril(C, -1).T #матрица затрат должна быть симметричной
np.fill_diagonal(C, 0, wrap=False)

P = np.random.randint(0, 2, (n, n)) #несуществующие ребра графа
P = np.tril(P) + np.tril(P, -1).T
#P = (P + P.T)/2
np.fill_diagonal(P, 1, wrap=False) 

for i in range(n):
    for j in range(n):
        if P[i][j] == 1:
            C[i][j] = 10

print('матрица затрат')
print(C)

C = torch.from_numpy(C).cuda()
P = torch.from_numpy(P).cuda()


матрица затрат
[[10 10 10 10 10  9  7  9 10  5  0 10 10  1  2 10 10 10 10  4 10 10 10  1
  10 10 10  2 10 10]
 [10 10  4 10  7 10  7  7  7 10  7  4 10 10 10  0  9  3 10 10 10 10  2 10
   7 10 10 10 10 10]
 [10  4 10  7  4 10 10  6  5 10 10 10  0 10  9  7 10 10 10  4  5 10  5 10
  10 10 10 10 10 10]
 [10 10  7 10 10  8 10 10  8 10 10  8 10 10  7  1 10  5  5  7 10  1 10  5
  10  8 10 10 10 10]
 [10  7  4 10 10  9 10 10 10  9 10 10 10  0 10 10 10 10  2  5 10 10  5 10
   5  1 10 10  6  5]
 [ 9 10 10  8  9 10  8  1 10  1 10  4 10  4 10  8 10 10  3 10 10 10  5  4
  10 10  0 10  5 10]
 [ 7  7 10 10 10  8 10 10  2  8 10  5  5 10  4 10  7  9 10 10 10 10 10  7
  10  7  2 10  3 10]
 [ 9  7  6 10 10  1 10 10  7 10 10 10  0  0  0 10  2 10  4 10 10 10  0 10
   9 10 10  0 10  6]
 [10  7  5  8 10 10  2  7 10  9  3 10  0 10 10 10  3  0  5  9  5 10  8 10
   7 10  6 10  2 10]
 [ 5 10 10 10  9  1  8 10  9 10  7  8 10  7 10  2 10  9 10 10  5 10 10  2
  10  4  5 10 10 10]
 [ 0  7 10 10 10 10 10 10  3  7 10 

In [0]:
mu1 =torch.FloatTensor([950.]).cuda()#3000
mu2 = torch.FloatTensor([475.]).cuda()#480
mu3 = torch.FloatTensor([1500.]).cuda()#50000
mu4 = torch.FloatTensor([2500.]).cuda()#15000
mu5 = torch.FloatTensor([2500.]).cuda()#50000

In [24]:
A = torch.FloatTensor([0.0001]).cuda() 
B = torch.FloatTensor([0.00001]).cuda()
D = torch.FloatTensor([0.00001]).cuda()

ex_number = np.around(n.mul(2/3), decimals=0) # 2/3 - кол-во исполнителей относительно общего числа работников
director = 0 #номер узла, соответствующего топ-менеджеру в иерархии
executors = random.sample(range(1,n), int(ex_number)) #случайно выбранные индексы исполнителей  
print(executors)

[27, 17, 3, 8, 25, 24, 16, 13, 18, 19, 9, 15, 5, 1, 2, 7, 10, 21, 11, 29]


In [0]:
weights = torch.zeros((n*n,n*n)).cuda()

In [0]:
def init_weights():
    start = time.time()
    '''x = torch.ByteTensor([0]).cuda()
    y= torch.ByteTensor([0]).cuda()
    i= torch.ByteTensor([0]).cuda()
    j= torch.ByteTensor([0]).cuda()'''
      
    for x in range(n):
        for i in range(n):
            for y in range(n):
                for j in range(n):
                    if x==y==i==j:
                        weights[n*x+i,n*y+j] = mu2
                    elif x==y and i==j and j!=x:
                        weights[n*x+i,n*y+j] = mu2.sub(mu4.sub(mu4))  
                    elif (x==y and i!=j and (j==x or i==y)) or (x!=y!=i!=j) or (x!=y and i==j and (j==x or i==y)):
                        weights[n*x+i,n*y+j] = 0.
                    elif x!=y and i!=j and j==x and i==y:
                        weights[n*x+i,n*y+j] =  mu4.add(mu4)  
                    elif x!=y and i!=j and (j==x or i==y):    
                        weights[n*x+i,n*y+j] = mu4
                    elif (x==y or i==j) and j!=x and i!=y:
                        weights[n*x+i,n*y+j] = -mu4
                     
                    #weights[n*x+i,n*y+j] = mu2.mul(torch.FloatTensor([kron(x, y)*kron(i, j)]).cuda()) - mu4.mul(torch.FloatTensor([kron(x, y)]).cuda()) - mu4.mul(torch.FloatTensor([kron(i, j)]).cuda()) + mu4.mul(torch.FloatTensor([kron(j, x)]).cuda()) + mu4.mul(torch.FloatTensor([kron(i, y)]).cuda())
                    #mu2*kron(x, y)*kron(i, j) - mu4*kron(x, y) - mu4*kron(i, j) + mu4*kron(j, x) + mu4*kron(i, y)
    
    finish = time.time()
    result = finish - start
    print("Время на инициализацию матрицы весов: " + str(result) + " секунд")
    return weights

In [29]:
init_weights()

Время на инициализацию матрицы весов: 59.05066418647766 секунд


tensor([[  475.,     0.,     0.,  ...,     0.,     0.,     0.],
        [    0.,   475., -2500.,  ...,     0.,     0.,     0.],
        [    0., -2500.,   475.,  ...,     0.,     0.,     0.],
        ...,
        [    0.,     0.,     0.,  ...,   475., -2500.,     0.],
        [    0.,     0.,     0.,  ..., -2500.,   475.,     0.],
        [    0.,     0.,     0.,  ...,     0.,     0.,   475.]],
       device='cuda:0')

In [0]:
def energy(m,OUT, ftype):
    #OUT = K.reshape(OUT, (n,n))
    #ftype = ftype + OUT
     
    E1 = torch.FloatTensor([0]).cuda() 
    E2 = torch.FloatTensor([0]).cuda()
    E3 = torch.FloatTensor([0]).cuda()
    E4 = torch.FloatTensor([0]).cuda()
    E5 = torch.FloatTensor([0]).cuda()
    
    for x in range(n):
        for i in range(n):
            if (i != x) and (x!=m and i!=director):
                E1 = E1.add(torch.mul(C[x][i], torch.mul(torch.div(1,torch.add(1,ftype[x][i])),OUT[x][i]))) 
                #E1 + C[x][i]*(1/(1+ftype[x][i]))*OUT[x][i]  
                E3 = E3.add(torch.mul(P[x][i],OUT[x][i]))
                #E3+P[x][i]*OUT[x][i]
            if i != x:
                E2 =  E2.add(torch.mul(OUT[x][i],torch.sub(1,OUT[x][i])))
                #E2 + OUT[x][i]*(1-OUT[x][i])
                 
                
    out1=torch.FloatTensor([0]).cuda()
    out2=torch.FloatTensor([0]).cuda()
    #square=0
    for x in range(n):
        for i in range(n):
            if i != x:
                out1 = out1.add(OUT[x][i])
                #out1+OUT[x][i]
                out2 = out1.add(OUT[i][x])
                #out2+OUT[i][x]
        #square = (out1-out2)*(out1-out2)
        E4 = E4.add(torch.mul(out1.sub(out2),out1.sub(out2)))
        #E4 + (out1-out2)*(out1-out2)
        
    E5 = torch.sub(1,OUT[m][director])
    #1-OUT[m][director]
    
    E =  torch.add(mu1.mul(E1),mu2.mul(E2))
    E = E.add(mu3.mul(E3))
    E = E.add(mu4.mul(E4))
    E = E.add(mu5.mul(E5))
    #torch.add(mu1.mul(E1),torch.add(mu2.mul(E2),torch.add(mu3.mul(E3),torch.add(mu4.mul(E4),torch.add(mu5.mul(E5)))))) 
    #mu1*E1 + mu2*E2 + mu3*E3 + mu4*E4 + mu5*E5
    
    #OUT = K.reshape(OUT, (n*n,1))
    return E